In [1]:
from google.colab import drive
drive.mount('/content/Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [2]:
!pip install glove_python
!pip install fasttext

     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700317 sha256=b89846f82bed622fa35fded6df4b4bc69f339d30f0c07352d60f0d298776f618
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2388479 sha256=e08a1f91142c2cf7f89e0b4f6628805f476bdddd36dea45858055cb5e9809da0
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
# #!git clone https://github.com/facebookresearch/fastText.git
# !cd fastText
# !sudo pip install fastText

Processing /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3/fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl


In [3]:
import argparse
import pprint
import gensim
from glove import Glove
from glove import Corpus

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from itertools import product

os.chdir('/content/Drive/My Drive/Colab Notebooks/2019_AI_CUP_LABELING/labeling_cowork_folder')
os.getcwd()

'/content/Drive/My Drive/Colab Notebooks/2019_AI_CUP_LABELING/labeling_cowork_folder'

In [0]:
import pandas as pd

dataset = pd.read_csv('raw_data/task1_trainset.csv', dtype=str)
# dataset.head()

In [0]:
# 句子有一些換行沒處理好
dataset["Abstract"] = dataset["Abstract"].str.replace("- ", "")

dataset.drop('Title',axis=1,inplace=True)
dataset.drop('Categories',axis=1,inplace=True)
dataset.drop('Created Date',axis=1, inplace=True)
dataset.drop('Authors',axis=1,inplace=True)

In [6]:
from sklearn.model_selection import StratifiedKFold, KFold

trainset_folds = []
trainset_folds = []

valset_folds = []
valset_folds = []

tmp_train = {}
tmp_val = {}

X = dataset["Abstract"]
y = dataset["Task 1"]

# 用GROUP K-FOLD分類別? 先改用K-Fold

K_folder = KFold(n_splits=10, random_state=42, shuffle=False)
for i, (train_index, val_index) in enumerate(K_folder.split(X, y)):

    tmp_train['Abstract'] = X[train_index]
    tmp_train['Label'] = y[train_index]

    tmp_val['Abstract'] = X[val_index]
    tmp_val['Label'] = y[val_index]

    # 有一個split會反過來 (Stratified K-Fold)
    # if i == 0:
    #   continue

    trainset_folds.append(pd.DataFrame(data=tmp_train))
    valset_folds.append(pd.DataFrame(data=tmp_val))

    # print("Stratified Train Index:", train_index)
    # print("Stratified Test Index:", test_index)
    # print("Stratified y_train:", y[train_index])
    # print("Stratified y_test:", y[test_index],'\n')

# train_test_split(dataset, test_size=0.2, random_state=42)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
def save_k_fold_data(data, name="train_data_fold", fold=10):
  """
  X_data: list of DF
  y_data: list of DF
  """
  for i in range(fold):
    data[i].to_csv("K_fold_data/" + name + "_" + str(i) + ".csv", index=None)
fold=10
# save_k_fold_data(data=trainset_folds, name="train_data_fold", fold=fold)
# save_k_fold_data(data=valset_folds, name="valid_data_fold", fold=fold)

In [0]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
trainset, validset = train_test_split(dataset, test_size=0.1, random_state=42)

trainset.to_csv('train_test_split_data/trainset.csv',index=False)
validset.to_csv('train_test_split_data/validset.csv',index=False)

### For Test Data

In [0]:
dataset = pd.read_csv('raw_data/task1_public_testset.csv', dtype=str)
dataset["Abstract"] = dataset["Abstract"].str.replace("- ", "")
dataset.drop('Title',axis=1,inplace=True)
dataset.drop('Categories',axis=1,inplace=True)
dataset.drop('Created Date',axis=1, inplace=True)
dataset.drop('Authors',axis=1,inplace=True)
dataset.to_csv('train_test_split_data/testset.csv',index=False)

In [0]:
import pandas as pd
from multiprocessing import Pool
from nltk.tokenize import word_tokenize

def tokenize_by_space(sentence):
  return sentence.split(' ')

def collect_words(data_path=None, n_workers=4, dataset=None):
    if dataset is None:
      df = pd.read_csv(data_path, dtype=str)
    else:
      df = dataset
      
        
    sent_list = []
    for i in df.iterrows():
        sent_list += i[1]['Abstract'].split('$$$')

    chunks = [
        ' '.join(sent_list[i:i + len(sent_list) // n_workers])
        for i in range(0, len(sent_list), len(sent_list) // n_workers)
    ]
    with Pool(n_workers) as pool:
        chunks = pool.map_async(word_tokenize, chunks)
        # chunks = pool.map_async(tokenize_by_space, chunks)
        words = set(sum(chunks.get(), []))

    return words

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
words = set()
# testset_w = pd.read_csv("train_test_split_data/testset.csv", index_col=None)
# trainset_w = pd.read_csv("train_test_split_data/trainset.csv", index_col=None)
# all_data = pd.concat([trainset_w, testset_w])
words |= collect_words('raw_data/task1_trainset.csv') # 只拿train的去訓練詞向量

# words |= collect_words(dataset=all_data)

In [0]:
words

{'MANet',
 'pre-trained',
 'successive',
 'msEPI',
 'bend',
 'Self-healing',
 'max-plus',
 'increasingly-popular',
 'fulfil',
 'Milner',
 '26.27',
 'life-cycle',
 'Lots',
 'RHA',
 'injuries',
 'Latvian',
 'subcortical',
 'MAL',
 'pharmacovigilance',
 'document',
 'fuse',
 'online-to-print',
 '1960',
 'parsing-by-detection',
 'differential-algebraic',
 'Historically',
 'free-choice',
 'CCCS',
 're-weight',
 'mel',
 'Services',
 'developers/testers',
 'Hama',
 'VOC',
 'PhDs',
 'adequate',
 '2K-wire',
 'MFD',
 'debase',
 'scatterers',
 'COCO-stuff',
 'shortcut',
 'canonicalizing',
 'hypothetic',
 'AUVs',
 'suffers',
 'textures',
 'opened',
 '1200',
 '45.8',
 'cleaning',
 'current-day',
 'impedes',
 'excited-state',
 '17.6',
 'attractor',
 'Citations',
 'MCFA',
 'necessitated',
 'libraries',
 'cater',
 'replica',
 'eras',
 'clutters',
 'Allen',
 'solutions',
 'smells',
 'overloads',
 'Earlier',
 'MODB',
 'Sphere',
 'life-saving',
 'dissect',
 'melodic',
 'analytically',
 'jobs',
 'Generali

In [0]:
import requests
from bs4 import BeautifulSoup
import json
import pickle

r = requests.get("https://brandirectory.com/rankings/global-500-2018") #將網頁資料GET下來
soup = BeautifulSoup(r.text, "html.parser") # 將網頁資料以html.parser
# sel = soup.select("a.tight-text") # 取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
# sel
text = soup.find_all("script", type="application/ld+json")[0]
contents_dict = json.loads(text.contents[0])
# type(contents_dict)
brand_names = [d["item"]["name"] for d in contents_dict["itemListElement"]]
brand_names[:6]

with open('other_data/dicitonary_Brands.pkl','wb') as f:
    pickle.dump(brand_names, f)

In [0]:
import string
import re

punctuations = ''.join([s for s in string.punctuation if s not in ('<', '>', '-')])

def word_preprocess(word):

  # remove punctuations
  word = word.translate(str.maketrans('', '', punctuations))

  # remove numbers
  if '-' not in word:
    word = re.sub(r'\d+', '', word)

  if '-' in word:
    if (np.sum([w.isdigit() for w in word]) / len(word)) > 0.8:
      word = '<numbers>'
    else:
      # pass
      word = '<cool-abr>'

  # remove single character
  if len(list(word)) < 1:
    word = ""

  # brands
  if word in brand_names:
    word = '<brand>'

  # proprietary 1
  # if len(word) > 1:
  #   if (list(word)[0] == '(') and (list(word)[-1] == ')'):
  #     word = '<abr>'

  # all uppercase
  # if len(word) > 1:
  if np.sum([w.isupper() for w in word]) >= 2:
    word = '<pry>'
  
  # lemmatization
  # lemma = nltk.wordnet.WordNetLemmatizer()
  # word = lemma.lemmatize(word)

  return word.lower()


In [0]:
from nltk.corpus import stopwords
from multiprocessing import Pool

with Pool() as pool:
  result = pool.map(word_preprocess, words)

words_pre = set(result)

print(stopwords.words('english'))
# words_pre = set([w.lower() for w in words_pre])
# words_pre = words_pre - set(stopwords.words('english'))

words_pre = words_pre - {''}
words_pre

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

{'successive',
 'bend',
 'mints',
 'digitally',
 'transported',
 'fulfil',
 'hospital',
 'trimaps',
 'paraconsistent',
 'injuries',
 'subcortical',
 'overcome',
 'starterconclusions',
 'assembled',
 'pharmacovigilance',
 'document',
 'fuse',
 'pyramids',
 'contracts',
 'refinements',
 'exert',
 'planning',
 'mel',
 'vowels',
 'andor',
 'easier',
 'adequate',
 'debase',
 'thorniness',
 'scatterers',
 'hypothetic',
 'shortcut',
 'canonicalizing',
 'underutilisation',
 'ns',
 'suffers',
 'textures',
 'opened',
 'virgili',
 'cleaning',
 'impedes',
 'inflexibility',
 'journals',
 'attractor',
 'necessitated',
 'libraries',
 'cater',
 'replica',
 'eras',
 'information',
 'scalarizations',
 'barral',
 'parliament',
 'downsampling',
 'clutters',
 'notebook',
 'solutions',
 'smells',
 'operational',
 'overloads',
 'mali',
 'modulethe',
 'storages',
 'wear',
 'parallelization',
 'dissect',
 'melodic',
 'transposed',
 'analytically',
 'jobs',
 'normalisations',
 'theasholding',
 'immunize',
 'evi

In [0]:
len(words_pre)

20901

In [0]:
PAD_TOKEN = 0
UNK_TOKEN = 1
word_dict = {'<pad>':PAD_TOKEN,'<unk>':UNK_TOKEN}
for word in words_pre:
    word_dict[word]=len(word_dict)

## 前處理句子



In [0]:
from tqdm import tqdm_notebook as tqdm

def sentence_preprocess(sentence):
  sentence = [word_preprocess(word) for word in word_tokenize(sentence)]
  return list(filter(lambda x: x != '', sentence))


def preprocess_sample(data):
  return [sentence_preprocess(sent) for sent in data['Abstract'].split('$$$')]


def preprocess_samples(dataset):
    processed = []
    for sample in tqdm(dataset.iterrows(), total=len(dataset)):
        processed.extend(preprocess_sample(sample[1]))

    return processed


def get_dataset(data_path, n_workers=4):
    """ Load data and return dataset for training and validating.

    Args:
        data_path (str): Path to the data.
    """
    dataset = pd.read_csv(data_path, dtype=str)

    results = [None] * n_workers
    with Pool(processes=n_workers) as pool:
        for i in range(n_workers):
            batch_start = (len(dataset) // n_workers) * i
            if i == n_workers - 1:
                batch_end = len(dataset)
            else:
                batch_end = (len(dataset) // n_workers) * (i + 1)
            
            batch = dataset[batch_start: batch_end]
            results[i] = pool.apply_async(preprocess_samples, args=(batch, ))
        
        # 有with基本上不用這個
        pool.close()
        pool.join()

    processed = []
    for result in results:
        processed += result.get()
    return processed

In [0]:
print('[INFO] Start processing trainset...')
train = get_dataset('train_test_split_data/trainset.csv', n_workers=4)
print('[INFO] Start processing validset...')
valid = get_dataset('train_test_split_data/validset.csv', n_workers=4)
print('[INFO] Start processing testset...')
test = get_dataset('train_test_split_data/testset.csv', n_workers=4)

[INFO] Start processing trainset...




[INFO] Start processing validset...




[INFO] Start processing testset...






### FastText

In [0]:
import fasttext

In [0]:
def sentence_to_string(sentence):
    return ' '.join(sentence)

with Pool() as pool:
    sent_list = pool.map(sentence_to_string, sentences)
    
sent_str_all = ''
for sent in sent_list:
    sent_str_all += " " + sent

In [0]:
with open("other_data/corpus_of_all_data.txt", "w", encoding='utf-8') as f:
    f.write(sent_str_all)


In [0]:
# Skipgram model :
model_fasttext = fasttext.train_unsupervised("other_data/corpus_of_all_data.txt", model='skipgram')
model_fasttext.save_model("other_data/fasttext.model.bin")
# or, cbow model :
# model = fasttext.train_unsupervised(sent_str_all, model='cbow')

In [0]:
model_fasttext = fasttext.load_model('other_data/fasttext.model.bin')
print(model_fasttext.words)
print(model_fasttext.get_word_vector("Wasserstein"))
print(model_fasttext.get_nearest_neighbors('Wasserstein'))

['the', 'of', 'and', 'a', 'to', '<cool-abr>', '<pry>', 'in', 'is', 'for', 'that', 'on', 'we', 'with', 'are', 'We', 'this', 'as', 'The', 'by', 'an', 'In', 'can', 'be', 'which', 'from', 'data', 'paper', 'model', 'This', 'our', 'network', 'using', 'results', 'problem', 'show', 'proposed', 'it', 'or', 'based', 'algorithm', 'such', 'method', 'have', 'learning', 'performance', 'approach', 'has', 'system', 'not', 'information', 'also', 'at', 'networks', 'used', 'new', 'these', 'two', 'propose', 'models', 'systems', 'between', 'different', 'their', 'methods', 'time', 'more', 'its', 'use', 'been', 'algorithms', 'number', 'both', 'Our', 'present', 'analysis', 'one', 'each', 'work', 'over', 'framework', 'image', 'into', 's', 'other', 'where', 'than', 'set', 'study', 'only', 'when', 'all', 'A', 'applications', 'large', 'problems', 'but', 'how', 'provide', 'neural', 'novel', 'well', 'To', 'training', 'design', 'features', 'However', 'some', 'first', 'many', 'images', 'most', 'process', 'demonstrate

AttributeError: ignored

#### 因為版本問題只能自己寫get_nearest_neighbors

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial

def get_cos_similarity(word1, word2):
    return cosine_similarity(X=model_fasttext.get_word_vector(word1).reshape(1, -1),
                               Y=model_fasttext.get_word_vector(word2).reshape(1, -1))

def get_nearest_neighbors(word1, n_words=10):
    # word1 = "biomedic"
    get_nearest_word_partial = partial(get_nearest_word, word1)

    with Pool() as pool:
        similarity = pool.map(get_nearest_word_partial, model_fasttext.words)

    sim_dict = {}
    for word, sim in zip(model_fasttext.words, similarity):
        sim_dict[sim[0][0]] = word
    
    return [(k, sim_dict[k]) for k in sorted(sim_dict.keys(), reverse=True)][:n_words]

In [0]:
get_nearest_neighbors(word1="biomedic")

[(0.95616984, 'biomedical'),
 (0.91933984, 'biomedicine'),
 (0.8204172, 'Biomedical'),
 (0.71463954, 'medical'),
 (0.71321994, 'bioinformatic'),
 (0.68795174, 'bioinformatics'),
 (0.6837375, 'cheminformatics'),
 (0.6719316, 'medicine'),
 (0.67089814, 'biomarker'),
 (0.66372675, 'biometrics')]

In [0]:
get_nearest_neighbors(word1="network")

[(1.0, 'network'),
 (0.84717506, 'networks'),
 (0.768269, 'subnetwork'),
 (0.7461433, 'subnetworks'),
 (0.6955533, 'nanonetworks'),
 (0.6651289, 'hypernetworks'),
 (0.66443825, 'multilayer'),
 (0.6617832, 'multilayered'),
 (0.6600789, 'backbone'),
 (0.64864516, 'backbones')]

## Glove

In [0]:
sentences = train # + valid + test
corpus_model = Corpus()
corpus_model.fit(sentences, window=10)
#corpus_model.save('corpus.model')
print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)

Dict size: 20000
Collocations: 2081964


In [0]:
glove = Glove(no_components=100, learning_rate=0.02)
# no_of_components — This is the dimension of the output vector generated by the GloVe
glove.fit(corpus_model.matrix, epochs=10,
          no_threads=1, verbose=True)
glove.add_dictionary(corpus_model.dictionary)

Performing 10 training epochs with 1 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
glove.save('other_data/glove_20901.model_v3')
corpus_model.save('other_data/corpus_20901.model_v3')

In [0]:
glove = Glove.load('other_data/glove.model')
corpus_model = Corpus.load('other_data/corpus.model')

### 相似詞

In [0]:
list(words)[:3]

['MANet', 'pre-trained', 'successive']

In [0]:
glove.most_similar('biometric', number=10)

[('secrecy', 0.9640390434019579),
 ('shannon', 0.9616899608258208),
 ('duration', 0.9609812760331041),
 ('maximal', 0.9609101738051686),
 ('outside', 0.9596632715656328),
 ('integrity', 0.9592439048212437),
 ('strengths', 0.9586225937687087),
 ('outage', 0.9585645886189805),
 ('air', 0.9585077174969163)]

In [0]:
glove.most_similar('reparameterisation', number=10)

Exception: ignored

### 詞向量矩陣

In [0]:
glove.word_vectors

array([[ 0.05940333,  0.16510622,  0.16847501, ...,  0.16029682,
        -0.16625296, -0.05430988],
       [-0.0476905 ,  0.00469257,  0.00936   , ...,  0.01129865,
        -0.00562537,  0.04169494],
       [ 0.01565656,  0.07076137,  0.08037127, ...,  0.08042861,
        -0.07332118, -0.04287636],
       ...,
       [ 0.00225355, -0.01174298, -0.01666196, ..., -0.01429752,
         0.0139943 , -0.00213089],
       [-0.00103333, -0.00641306, -0.01156102, ..., -0.01065586,
         0.00921793, -0.00077878],
       [-0.00720772, -0.01592166, -0.02250754, ..., -0.02105807,
         0.0161561 ,  0.0059752 ]])

In [0]:
glove.word_vectors[glove.dictionary['Wasserstein']]

array([-4.76905006e-02,  4.69257318e-03,  9.35999567e-03, -4.52076284e-03,
        3.97221414e-03,  2.84903076e-03,  1.13711726e-02, -5.69430104e-03,
        6.76210759e-03,  9.78754447e-03,  5.03885295e-03,  2.97957475e-02,
        6.93344339e-02, -1.21090110e-02, -1.31525824e-02, -1.17355498e-02,
       -6.68133935e-04,  6.57924849e-02, -1.18996313e-02, -9.51533935e-03,
        8.39242489e-03,  1.61236451e-02,  3.76153695e-03, -9.03720932e-03,
        1.03737222e-02, -9.27434723e-03, -1.25528517e-02, -3.50281200e-02,
       -1.17345823e-02, -1.34235237e-02, -1.03796791e-02, -1.61522432e-02,
        1.84651399e-03, -6.55436445e-06,  1.30355010e-02, -2.54190620e-04,
       -1.22425643e-02, -5.49893474e-03,  4.71289910e-03, -5.07278317e-03,
       -1.47964093e-03,  9.25667931e-03, -6.02807674e-03,  5.91542556e-02,
        8.46933922e-04,  8.05122550e-03,  3.24639235e-04, -5.81661767e-03,
        9.68991727e-03, -8.03256475e-02, -9.91408529e-03, -1.93195949e-02,
        4.60665879e-03, -

In [0]:
# corpus_model.matrix.todense().tolist() # co-concurrence matrix 太大print出來會掛

NameError: ignored

In [0]:
train_folds=[]
valid_folds=[]

for i in range(fold):
  print('[INFO] Start processing trainset...')
  train_folds.append(get_dataset("K_fold_data/" + "train_data_fold_" + str(i) + ".csv", word_dict, n_workers=4))
  print('[INFO] Start processing validset...')
  valid_folds.append(get_dataset("K_fold_data/" + "valid_data_fold_" + str(i) + ".csv", word_dict, n_workers=4))